In [1]:
import pandas as pd
import numpy as np

In [2]:
# import data
productDF = pd.read_csv('../data/lazada/clean/product.csv')
reviewDF = pd.read_csv('../data/lazada/clean/review.csv')

In [3]:
# aggregate
productRatingDF = reviewDF[['sku', 'rating']].copy()
productRatingDF['rating_count'] = productRatingDF['rating']
productRatingDF['rating_mean'] = productRatingDF['rating']
productRatingDF['rating_sum'] = productRatingDF['rating']
productRatingDF = productRatingDF[['sku', 'rating_sum', 'rating_count', 'rating_mean']] \
    .groupby('sku') \
    .agg({
        'rating_sum': sum,
        'rating_count': len,
        'rating_mean': np.mean
    }) \
    .sort_values('rating_sum', ascending=False) \
    .reset_index()

In [4]:
# normalize rating
productRatingDF['rating_sum_rank'] = productRatingDF['rating_sum'].rank(pct=True)
productRatingDF['rating_count_rank'] = productRatingDF['rating_count'].rank(pct=True)
productRatingDF['rating_mean_rank'] = productRatingDF['rating_mean'].rank(pct=True)

# weighted sum percentile
w_count = 1
w_sum = 1
w_mean = 1
productRatingDF['final_score'] = \
    w_sum * productRatingDF['rating_sum_rank'] \
    + w_mean * productRatingDF['rating_mean_rank'] \
    + w_count * productRatingDF['rating_count_rank']

In [6]:
# explore ranking
productDF[['sku', 'name']] \
    .merge(productRatingDF.sort_values('final_score', ascending=False)) \
    .sort_values('final_score', ascending=False) \
    .drop(['sku'], axis=1) \
    .head(10)

,sku,name,rating_sum,rating_count,rating_mean,rating_sum_rank,rating_count_rank,rating_mean_rank,final_score
4205,WA044FAAA44MN8ANTH,WARRIX SPORT กางเกงวอร์ม กีฬา WP-1707 สีดำ,509,108,4.712963,0.990597,0.986951,0.867972,2.845519
847,FA885ELAA3JG9WANTH,Fantech Gaming Mousepad แผ่นรองเมาส์แบบสปีด สำ...,329,68,4.838235,0.976972,0.966129,0.897141,2.840242
3334,DE261ELAA1V7PZANTH,"Dell MS116 USB Optical Mouse 1000dpi, 1.8m (Bl...",477,102,4.676471,0.989637,0.985320,0.864709,2.839666
1963,KI572ELAA1ONLDANTH,Kingston Micro SD Class 10 U1 80/10 MB/s - 16G...,894,194,4.608247,0.996162,0.995394,0.835348,2.826905
1961,SA862ELAA1JXU7ANTH,Sandisk MicroSD Ultra Class 10 48MB/S - 32GB(S...,6141,1343,4.572599,1.000000,1.000000,0.825178,2.825178
1182,CA175ELAA23MW9ANTH,Canon Lens EF 50mm f/1.8 STM,558,120,4.650000,0.992036,0.988582,0.841393,2.822011
1967,SA862ELAA1JXU6ANTH,Sandisk MicroSD Ultra Class 10 48MB/S - 16GB(S...,3654,806,4.533499,0.999616,0.999616,0.814815,2.814047
382,AU889ELAA666RTANTH,[สายหนา 4 mm] สายชาร์จ Aukey Quick Charge 3.0+...,270,56,4.821429,0.966609,0.954519,0.892727,2.813855
248,EL684ELBHMFOANTH,Eloop E14 Power Bank 20000mAh (สีดำ) ฟรี ซองกำ...,1440,318,4.528302,0.998657,0.998273,0.813855,2.810785
942,DE261ELAA2Q5M4ANTH,Dell Full HD LCD Monitor 23.8” U2414H (Free! D...,910,201,4.527363,0.996546,0.995970,0.813663,2.806179


In [ ]:
# productRatingDF.to_csv('popular/util.csv', index=False)